In [ ]:
import pandas as pd
import folium

# Folium 지도표시
# 위도,경도로 지도에 표시

# 서울지도
seoul_map = folium.Map(location=[37.55, 126.98], zoom_start=12)
seoul_map

In [ ]:
# 지도를 HTML 파일로 저장
seoul_map.save('./test.html')

In [ ]:
# 서울지도, # 옵션추가
seoul_map = folium.Map(location=[37.55, 126.98],
                       tiles='Stamen Terrain',
                       #tiles='Stamen Toner',
                       #tiles='OpenStreetMap',      #default
                       zoom_start=12)
seoul_map

In [6]:
seoul_map.save('./test2.html')

In [21]:
# .. 서울 대학교 위치
# 대학교 리스트를 데이터 프레임으로 변환
df = pd.read_excel('/content/서울지역 대학교 위치.xlsx', engine='openpyxl')
df.set_index(['Unnamed: 0'],inplace=True)
df.head()

,위도,경도
Unnamed: 0,,
KAIST 서울캠퍼스,37.592573,127.046737
KC대학교,37.548345,126.854797
가톨릭대학교(성신교정),37.585922,127.004328
가톨릭대학교(성의교정),37.499623,127.006065
감리교신학대학교,37.567645,126.961610


In [22]:
# 맵 불러오기
seoul_map = folium.Map(location=[37.55, 126.98], tiles='Stamen Terrain', zoom_start=12)

In [23]:
# 대학교 위치정보를 Maker로 표시
for name, lat, lon in zip(df.index, df.위도, df.경도):
    folium.Marker( [lat, lon], popup=name ).add_to(seoul_map) # popup: 팝업창, 마우스 클릭 정보표시
seoul_map

In [24]:
# 마커 변경
# 맵 불러오기
seoul_map2 = folium.Map(location=[37.55, 126.98], tiles='Stamen Terrain', zoom_start=12)
for name, lat, lon in zip(df.index, df.위도, df.경도):
    folium.CircleMarker( [lat, lon], 
                       radius=10, 
                       color='brown',    # 원 둘레색
                       fill=True, 
                       fill_color='coral',
                       fill_opacity=0.7, 
                       popup=name).add_to(seoul_map2) 
seoul_map2

In [31]:
# .. 경기도 인구데이터(xlsx) 및 행정구역경계(json)
# 인구 데이터 로드
df1_pop = pd.read_excel('/content/경기도인구데이터.xlsx', engine='openpyxl', index_col='구분')
df1_pop.columns = df1_pop.columns.map(str)
df1_pop.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
구분,,,,,,,,,,,
수원시장안구,287474,285803,290798,293692,290999,291757,300908,301196,299016,296479,293828
수원시권선구,310553,308177,304097,306783,321176,332633,331773,339835,351053,358393,365653
수원시팔달구,216945,213235,219833,216503,209772,206794,204805,203479,199180,198515,193311
수원시영통구,252730,260210,258421,260557,266542,289074,310671,329718,335375,340654,349836
성남시수정구,256744,248452,242161,241070,236123,233997,228167,223539,219531,232841,236932


In [41]:
# 시군구 경계 정보를 가진 geo-json 파일 불러오기
import json
geo_path = '/content/경기도행정구역경계.json'
try: 
    #df1_geo = pd.read_json(geo_path)
    df1_geo = json.load(open(geo_path, encoding='utf-8'))
except:
    df1_geo = json.load(open(geo_path, econding='utf-8-sig'))   # return dict

print(type(df1_geo))

<class 'dict'>


In [38]:
g_map = folium.Map(location=[37.5502, 126.982], title='Stamen Terrain', zoom_start=9)

In [ ]:
# 출력할 연도를 선택
year = '2017'
# 단계구분도(Choropleth) 단계구분도 표시하기
folium.Choropleth(geo_data = df1_geo, data=df1_pop[year],
                  columns=[df1_pop.index, df1_pop[year]],
                  fill_color='YlOrRd', fill_opacity=0.7,
                  line_opacity=0.3,
                  threshold_case=[10000,100000,300000,500000,7000000],
                  key_on='feature.properties.name').add_to(g_map)
g_map

In [60]:
g_map.save('./gyonggi_population' + year + '.html')